<a href="https://colab.research.google.com/github/RERobbins/data_science_266_sandbox/blob/main/3_QA_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An Introduction to Question Answering Using Retrieval Augmented Generation

In our introduction to vector databases, we noted that retrieval augmented generation or "RAG" is used in tasks like question answering. With RAG, a retrieval component first selects a set of relevant documents or passages from a larger corpus, and then a generation component generates the final response based on the selected information. This approach aims to combine the accuracy of retrieval with the flexibility of generation.

This notebook builds on our work with vector databases.  We will take a query from the user and present it to the vector database. Then, we present that query and the relevant documents to the generative model to generate an answer.

We will make use of LangChain to coordinate this activity and explore model prompting.

We will answer questions about the privacy policies we have been working with.

# Generative Models, Embedding Models and Vector Databases

This notebook assumes that you use the same generative model throughout.  You will rely on the API keys you needed to work on the Embeddings notebook.

This notebook uses the same SentenceTransformer embedding model we used in the Vector Database notebook.  As was the case with that notebook, you are free to select another embedding model.

Finally, this notebook will use the Qdrant vector database.  You are, of course, free to experiment with other vector databases as we discussed before.

# Setup

## Environment Related Helpers

This portion of the notebook includes `install_if_needed` which will install a single package or list of packages with `pip` only if necessary, and `running_in_colab` a predicate that returns `True` if the notebook is running in Google Colab.

In [1]:
import importlib
import warnings
import textwrap

warnings.filterwarnings("ignore", category=UserWarning)


def install_if_needed(package_names):
    """
    Install one or more Python packages using pip if they are not already installed.

    Args:
        package_names (str or list): The name(s) of the package(s) to install.

    Returns:
        None
    """
    if isinstance(package_names, str):
        package_names = [package_names]

    for package_name in package_names:
        try:
            importlib.import_module(package_name)
            print(f"{package_name} is already installed.")
        except ImportError:
            !pip install --quiet {package_name}
            print(f"{package_name} has been installed.")


def running_in_colab():
    """
    Check if the Jupyter Notebook is running in Google Colab.

    Returns:
        bool: True if running in Google Colab, False otherwise.
    """
    try:
        import google.colab

        return True
    except ImportError:
        return False

## Mount Google Drive

By default, the data you create in Google Colaboratory does not persist from session to session.  Each session runs in a virtual machine and when that machine goes away, so does your data.  If you want your data to persist, you must store it outside the virtual machine. Google Drive can be used for that purpose.  We use it later in this notebook to store the OpenAI and Cohere API keys.

In [2]:
if running_in_colab():
    from google.colab import drive

    drive.mount("drive")

Mounted at drive


## API Keys

In [3]:
install_if_needed("python-dotenv")

python-dotenv has been installed.


In [4]:
import os
import getpass

from dotenv import load_dotenv, find_dotenv

def env_file_path(
    colab_path="/content/drive/MyDrive/.env", other_path=f"{find_dotenv()}"
):
    """
    Returns the appropriate file path for the environment variables file (.env) based on the execution environment.

    This function is designed to determine the correct path for the environment variables file
    depending on whether the code is running in Google Colab or in a different environment.

    Args:
        colab_path (str, optional): The file path for the environment variables file in Google Colab.
            Default is '/content/drive/MyDrive/.env'.

        other_path (str, optional): The file path for the environment variables file in other environments.
            Default is '/workspace/.env'.

    Returns:
        str: The file path for the environment variables file (.env).
    """

    return colab_path if running_in_colab() else other_path

In [5]:
load_dotenv(env_file_path())
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
COHERE_API_KEY = os.environ["COHERE_API_KEY"]

## Langchain

In [6]:
install_if_needed("langchain")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
langchain has been installed.


## GPU Support (Optional)

In [7]:
import tensorflow as tf

print("GPU Available:", tf.config.list_physical_devices("GPU"))

GPU Available: []


In [8]:
install_if_needed("torch")
import torch

print("CUDA Available:", torch.cuda.is_available())

torch is already installed.
CUDA Available: False


## Embeddings

Instantiate the embeddings model.  

In [9]:
packages = [
    "openai",
    "cohere",
    "tiktoken",
    "transformers",
    "sentence_transformers",
]

install_if_needed(packages)

import openai, tiktoken

import cohere
from langchain.chat_models import ChatOpenAI
from langchain.llms import Cohere

from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.8 MB/s eta 0:00:00a 0:00:01
openai has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 40.3 MB/s eta 0:00:00
cohere has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.8 MB/s eta 0:00:00
tiktoken has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.9 MB/s eta 0:00:00
transformers has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.0 MB/s eta 0:00:00
sentence_transformers has been installe

In [10]:
st_model_name = "multi-qa-mpnet-base-cos-v1"
st_embeddings_model = HuggingFaceEmbeddings(model_name=st_model_name)
st_tokenizer = AutoTokenizer.from_pretrained(f"sentence-transformers/{st_model_name}")

embeddings_model = st_embeddings_model

## Vector Database

In [11]:
install_if_needed("qdrant-client")
from langchain.vectorstores import Qdrant

import qdrant_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.24.1 which is incompatible.
qdrant-client has been installed.


# Generative Model

OpenAI trial accounts expire after three months and provide access to `gpt-3.5-turbo` but not `gpt-4`.  Paid OpenAI accounts permit use of `gpt-4` as well and do not expire.  Cohere trial accounts do not expire, but the API rate limiting is more significant than OpenAI trial account rate limiting.

Set the `LLM` variable below to reflect the generative model you want to use.  

The results in most of the examples below will vary with your choice.


In [12]:
# llm = Cohere(model="command", temperature=0)
LLM = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# llm = ChatOpenAI(model="gpt-4", temperature=0)

# Load Documents, Split Into Chunks, Create Vector Database

**If you saved your Qdrant database when you worked on the vector database notebook you can skip this section and use the Load Persistent Vector Database section below.**

Otherwise, we use the following cells to download the privacy policies we have been working with and split them into chunks to be stored in the vector database.

In [ ]:
install_if_needed(["pypdf", "unstructured"])

from langchain.document_loaders import PyPDFLoader, UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.1/271.1 kB 4.8 MB/s eta 0:00:00a 0:00:01
pypdf has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 10.7 MB/s eta 0:00:00
unstructured has been installed.


In [ ]:
import pandas as pd

policy_data = [
    ("Apple",
     "Privacy Policy",
     "https://www.apple.com/legal/privacy/pdfs/apple-privacy-policy-en-ww.pdf",
    ),
    ("Cohere", "Privacy Policy", "https://cohere.com/privacy"),
    ("Google",
     "Privacy Policy",
     "https://static.googleusercontent.com/media/www.google.com/en//intl/en/policies/privacy/google_privacy_policy_en.pdf",
    ),
    ("Hugging Face", "Privacy Policy", "https://huggingface.co/privacy"),
    ("Meta",
     "Privacy Policy",
     "https://about.fb.com/wp-content/uploads/2022/07/Privacy-Within-Metas-Integrity-Systems.pdf",
    ),
    ("Threads", "Privacy Policy", "https://terms.threads.com/privacy-policy"),
    ("TikTok",
     "Privacy Policy",
     "https://www.tiktok.com/legal/page/us/privacy-policy/en",
    ),]

columns = ["organization", "title", "url"]

policy_df = pd.DataFrame(policy_data, columns=columns)

In [ ]:
def get_chunks(url, organization, title, chunk_size=2000, chunk_overlap=500):
    """
    This function takes a url to an organization's web page, organization name,
    and document title and returns chunks constructed from the target url.
    The function adds the url, the organization name and the document title
    as metadata to the chunks.

    Parameters:
    url (string): Target page.
    organization (string): Organization name.
    title: Document title.
    chunk_size (int, optional): Chunk size, default is 2000 characters.
    chunk_overlap (int, optional): Chunk overlap, default is 500 characters.

    Returns:
    list of chunks
    """

    # Use PyPDFLoader for pdf targets, otherwise UnstructuredURLLoader
    if os.path.splitext(url)[1] == ".pdf":
        loader = PyPDFLoader(url)
    else:
      loader = UnstructuredURLLoader([url])

    documents = loader.load()
    for document in documents:
        metadata = document.metadata
        metadata["url"] = url
        metadata["organization"] = organization
        metadata["title"] = title
        if metadata.get("page", None) is not None:
            metadata["page"] += 1

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    return text_splitter.split_documents(documents)


def explore_documents(documents):
    block_indent = "   "
    metadata = documents[0].metadata
    content = documents[0].page_content[:300] + ". . ."
    print(f"{metadata['organization']} {metadata['title']} {len(documents)} chunks")
    print("Truncated First chunk:")
    print(
        textwrap.fill(
            content,
            initial_indent=block_indent,
            subsequent_indent=block_indent,
            replace_whitespace=True,
        )
    )
    print()

In [ ]:
chunks = []

for row in policy_df.itertuples(index=False):
    policy_chunks = get_chunks(row.url, row.organization, row.title)
    explore_documents(policy_chunks)
    chunks += policy_chunks

Apple Privacy Policy 18 chunks
Truncated First chunk:
   Apple Privacy Policy Apple’s Privacy Policy describes how Apple
   collects, uses, and shares your personal data. Updated December 22,
   2022 In addition to this Privacy Policy, we provide data and
   privacy information embedded in our products and certain features
   that ask to use your personal data. This . . .



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Cohere Privacy Policy 10 chunks
Truncated First chunk:
   Products  For Developers  For Business  Pricing  Blog  Company  Try
   now  Cohere Privacy Policy  Last Update: Aug 4, 2023  Cohere Inc.
   (“Cohere”) values and respects your privacy. We have prepared this
   privacy policy to explain the manner in which we collect, use, and
   disclose personal information th. . .

Google Privacy Policy 20 chunks
Truncated First chunk:
   Privacy Policy Last modified: December 18, 2017 ( view archived
   versions ) (The hyperlinked examples are available at the end of
   this document.) There are many different ways you can use our
   services – to search for and share information, to communicate with
   other people or to create new content. Wh. . .

Hugging Face Privacy Policy 12 chunks
Truncated First chunk:
   Terms of Service  Privacy Policy  Content Policy  Code of Conduct
   Hugging Face Privacy Policy  🗓 Effective Date: March 28, 2023  We
   have implemented this Privacy Policy because

In [ ]:
print (f"There are {len(chunks)} chunks.")

There are 140 chunks.


In [ ]:
%%time

collection_name = "my_collection"

vectordb = Qdrant.from_documents(
    documents = chunks,
    embedding = embeddings_model,
    location = ":memory:",
    collection_name = collection_name
    )

CPU times: user 5.9 s, sys: 890 ms, total: 6.79 s
Wall time: 12.1 s


In [ ]:
# Confirm that we have the same number of vectors in the vector database as we have chunks.

assert vectordb.client.get_collection(collection_name).vectors_count == len(chunks)

# Load Persistent Vector Database

**If you saved your Qdrant database when you worked on the vector database notebook you can use this section instead of the Load Documents, Split Into Chunks, Create Vector Database section above.**

If you execute this code block more than once in a session you are likely to get an error indicating that your vector database is already accessed by another instance of Qdrant client and that if you require concurrent access, you should use Qdrant server instead.

The prior section should always work in lieu of this section.

In [13]:
collection_name = "my_collection"
qdrant_database_location = "/content/drive/MyDrive/my_qdrant"

client = qdrant_client.QdrantClient(path=qdrant_database_location)

vectordb = Qdrant(client=client,
                   collection_name=collection_name,
                   embeddings=embeddings_model,)

In [14]:
assert vectordb.client.get_collection(collection_name).vectors_count == 140

# Prompting a Model

Before we introduce working with the vector database, let's experiment with some simple model prompts.  We will pass a string to Cohere's `command` model, which is its default generative model, and see how it responds.  We will do the same thing with OpenAI's `chatgpt-3.5`.

The responses are based on information the model was trained on.  We don't know if it they are accurate.  Sources are not presented.  The Cohere response for Threads doesn't seem to relate to the Threads social media platform and the OpenAI model isn't able to fine Threads information.

In [15]:
from langchain.schema import HumanMessage

cohere_llm = Cohere(model="command", temperature=0)
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [16]:
query = "Does Apple sell my personal data?"
result = cohere_llm(query)
print(textwrap.fill(result.strip()))

Apple has a strong commitment to protecting the privacy of its
customers. The company does not sell personal data to third parties.
However, it does collect and use data to provide services and improve
products.  For example, Apple collects data about how people use their
devices, such as which apps are used and how often. This data is used
to improve the user experience and develop new features.  Apple also
collects data about customers' purchasing habits, which is used to
improve marketing and develop new products.  In addition, Apple
collects data about customers' location, which is used to provide
location-based services and improve maps.  Apple's commitment to
privacy is a key selling point for many customers. The company's
privacy policies are designed to protect customers' personal data and
ensure that it is used only for legitimate purposes.


In [17]:
query = "Does Threads sell my personal data?"
result = cohere_llm(query)
print(textwrap.fill(result.strip()))

Threads does not sell your personal data. We are committed to
protecting your privacy and will never share your personal information
with third parties without your permission.  We may collect and use
your personal information for the following purposes:  - To provide
you with the services you request - To improve our services and
products - To communicate with you about our services and products -
To protect our rights and property - To comply with legal obligations
We will never share your personal information with third parties
without your permission, except as required by law. We will never sell
your personal information to third parties.  If you have any questions
or concerns about how we use your personal information, please contact
us at support@threadscanada.com.


In [18]:
messages = [HumanMessage(content="Does Apple sell my personal data?")]
print(textwrap.fill(openai_llm(messages).content))

Apple has a strong commitment to privacy and has stated that it does
not sell personal data to third parties. Apple's business model
primarily relies on selling hardware, software, and services rather
than monetizing user data. However, it is important to note that Apple
does collect and use certain user data for various purposes, such as
improving its products and services, but it is typically anonymized
and aggregated to protect user privacy.


In [19]:
messages = [HumanMessage(content="Does Threads sell my personal data?")]
print(textwrap.fill(openai_llm(messages).content))

As an AI language model, I don't have access to specific company
policies or practices. However, it is important to note that I am
developed by OpenAI and designed to respect user privacy and
confidentiality. My primary function is to provide information and
answer questions to the best of my knowledge and abilities. If you
have concerns about data privacy, it is recommended to review the
privacy policy of Threads or contact the company directly for more
information.


# LangChain PromptTemplate

A prompt template is a reproducible way to generate prompts. It's essentially a text string that can take in a set of parameters from the end user and generate a prompt accordingly.  Let's shift to LangChain chains by using the simplest of templates.  In these examples, we use the large language model you selected above.  Remember, the model is generating responses based on its training data.

In [20]:
from langchain import PromptTemplate, LLMChain

In [21]:
template = """Question: {question} Answer:"""
prompt = PromptTemplate(template=template, input_variables=["question"])

chain = LLMChain(prompt=prompt, llm=LLM)

Let's inspect the prompt included inside the chain.

In [22]:
chain.prompt

PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='Question: {question} Answer:', template_format='f-string', validate_template=True)

In [23]:
query = "Does Apple sell my personal data?"
print(textwrap.fill(chain.run(query)).strip())

No, Apple does not sell your personal data. Apple has a strong
commitment to privacy and has implemented various measures to protect
user data.


In [24]:
query = "Does Threads sell my personal data?"
print(textwrap.fill(chain.run(query)).strip())

No, Threads does not sell your personal data.


# LangChain RetrievalQA Chain

Now we introduce our vector database and the LangChain Retrieval QA chain, a chain for question answering against a database of information.  We will also supply our own prompt. It would be reassuring if sources were identified.

In [25]:
from langchain.chains import RetrievalQA

In [26]:
template = """Use the following pieces of context to answer the question at the end.
Your answer should be as concise as possible and ideally not more than one sentence.
If you don't know the answer, just say that you don't know.

{context}

Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

chain = RetrievalQA.from_chain_type(
    LLM, retriever=vectordb.as_retriever(), chain_type_kwargs={"prompt": prompt}
)

Before we call the chain, let's inspect our template and the retriever it includes.

In [27]:
print(chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end.
Your answer should be as concise as possible and ideally not more than one sentence.
If you don't know the answer, just say that you don't know.

{context}

Question: {question}

Answer:


In [28]:
chain.retriever

VectorStoreRetriever(tags=['Qdrant', 'HuggingFaceEmbeddings'], metadata=None, vectorstore=<langchain.vectorstores.qdrant.Qdrant object at 0x78a04b4cb2b0>, search_type='similarity', search_kwargs={})

In [29]:
query = "Does Apple sell my personal data?"
result = chain.run(query)
print(textwrap.fill(result.strip()))

No, Apple does not sell your personal data.


In [30]:
query = "Does TikTok sell my personal data?"
result = chain.run(query)
print(textwrap.fill(result.strip()))

No.


Let's use the same prompt and get source documents too.

In [31]:
chain = RetrievalQA.from_chain_type(
    llm=LLM,
    retriever=vectordb.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True,
)

In [32]:
query = "Does Apple sell my personal data?"
result = chain(query)

Now, instead of returning a string, the result is a dictionary with three keys, `query`, `result` and `source_documents`.

In [33]:
result.keys()

dict_keys(['query', 'result', 'source_documents'])

In [34]:
print(textwrap.fill(result["result"].strip()))

No, Apple does not sell your personal data.


Let's examine the `organization` field for the source documents from that result.

In [35]:
[source.metadata["organization"] for source in result["source_documents"]]

['Apple', 'Apple', 'Apple', 'Apple']

Let's ask a general question and then say we only care about Apple and Hugging Face.  

In [36]:
query = "Do companies use cookies?  I only care about Apple and Hugging Face."
result = chain(query)

In [37]:
print(textwrap.fill(result["result"].strip()))

Yes, both Apple and Hugging Face use cookies.


In [38]:
[source.metadata["organization"] for source in result["source_documents"]]

['Apple', 'Hugging Face', 'Apple', 'Hugging Face']

Let's ask about Microsoft.  Remember, we have not loaded the Microsoft policy.  Notice that our answer only talks about Apple, our source documents include documents from Hugging Face and Threads, which aren't relevant.  This really isn't a great answer and the sources used do not inspire confidence.

In [39]:
query = "Do companies use cookies?  I only care about Apple, Cohere and Microsoft."
result = chain(query)

In [40]:
print(textwrap.fill(result["result"].strip()))

Yes, Apple uses cookies.


In [41]:
[source.metadata["organization"] for source in result["source_documents"]]

['Apple', 'Apple', 'Hugging Face', 'Threads']

This seems like a good place to make use of our metadata.  LangChain provides one way to do that, which we will explore in the next notebook in this series.